In [1]:
import torch
import networkx as nx
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import pandas as pd

# Device Configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Initialize GPT-2 Model
def initialize_model():
    """Initialize the tokenizer and GPT-2 model."""
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2", padding_side="left")  # Set padding_side to 'left'
    model = GPT2LMHeadModel.from_pretrained("gpt2").half()  # Load GPT-2 in half-precision
    model.to(device)  # Ensure it’s on the correct device

    # Set pad_token (GPT-2 doesn't have one by default, so we use eos_token as pad_token)
    tokenizer.pad_token = tokenizer.eos_token
    
    return tokenizer, model

# Create Knowledge Graph from CSV
def create_knowledge_graph_from_csv(csv_path):
    """Create a knowledge graph from a CSV file."""
    graph = nx.Graph()

    # Read CSV file
    data = pd.read_csv(csv_path)

    for _, row in data.iterrows():
        idea = row['Ideas']
        themes = [row['Theme a'], row['Theme-b'], row['Theme-c']]

        # Add nodes for the idea and its themes
        graph.add_node(idea)
        for theme in themes:
            if pd.notna(theme):
                graph.add_node(theme)
                graph.add_edge(idea, theme, relationship="relates_to")
    
    return graph

# Retrieve Knowledge from Graph
def retrieve_from_graph(graph, query):
    """Retrieve relevant information from the knowledge graph based on the input query."""
    relevant_nodes = set()
    relevant_edges = set()

    print(f"Debug: Searching for relevant nodes and edges for query: {query}")
    for node in graph.nodes:
        if any(keyword.lower() in query.lower() for keyword in node.split()):
            print(f"Debug: Found relevant node: {node}")
            relevant_nodes.add(node)

    for edge in graph.edges(data=True):
        if edge[0] in relevant_nodes or edge[1] in relevant_nodes:
            print(f"Debug: Found relevant edge: {edge}")
            relevant_edges.add(edge)

    return list(relevant_nodes), list(relevant_edges)

# Generate Context-Aware Responses
def generate_with_graph_knowledge(query, model, tokenizer, graph):
    """Generate a response based on a query and relevant knowledge from the graph."""
    relevant_nodes, relevant_edges = retrieve_from_graph(graph, query)

    # If no knowledge is found, provide a fallback message
    if not relevant_nodes and not relevant_edges:
        context = f"Query: {query}\nRelevant Knowledge: No relevant knowledge found in the graph."
    else:
        context = f"Query: {query}\nRelevant Knowledge:\n"
        for node in relevant_nodes:
            context += f" - Node: {node}\n"
        for edge in relevant_edges:
            context += f" - Edge: {edge[0]} ({edge[2]['relationship']}) {edge[1]}\n"

    # Use GPT-2 to generate a response
    inputs = tokenizer(context, return_tensors="pt", padding=True, truncation=True).to(device)

    outputs = model.generate(
        **inputs, 
        max_new_tokens=200, 
        num_return_sequences=1, 
        pad_token_id=tokenizer.pad_token_id
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Save and Load Knowledge Graph
def save_graph(graph, filepath="knowledge_graph.pkl"):
    """Save the knowledge graph to a file."""
    import pickle
    with open(filepath, "wb") as f:
        pickle.dump(graph, f)

def load_graph(filepath="knowledge_graph.pkl"):
    """Load the knowledge graph from a file."""
    import pickle
    with open(filepath, "rb") as f:
        return pickle.load(f)

# Main Processing Function
def main():
    """Main processing loop."""
    # Filepath to save/load the graph
    save_path = "knowledge_graph.pkl"
    csv_path = "151_ideas_updated.csv"

    # Load or create the knowledge graph
    try:
        graph = load_graph(save_path)
        print("Knowledge graph loaded successfully.")
    except FileNotFoundError:
        graph = create_knowledge_graph_from_csv(csv_path)
        save_graph(graph, save_path)
        print("Knowledge graph created and saved.")

    # Initialize the model
    tokenizer, model = initialize_model()

    # Example queries
    queries = [
        "How can mindfulness improve well-being and inner beauty?",
        "What is the relationship between pragmatism and philosophy?",
        "Explain the connection between meditation and personal growth.",
        "How does Nietzsche's view on intention relate to modern life?",
        "What role does beauty play in rational decision-making?"
    ]

    # Generate responses for each query
    for query in queries:
        print(f"Query: {query}")
        response = generate_with_graph_knowledge(query, model, tokenizer, graph)
        print(f"Response:\n{response}\n")

# Example Execution
if __name__ == "__main__":
    try:
        main()  # Process the queries and generate responses
    except Exception as e:
        print(f"An error occurred: {e}")


Using device: cpu
Knowledge graph loaded successfully.
Query: How can mindfulness improve well-being and inner beauty?
Debug: Searching for relevant nodes and edges for query: How can mindfulness improve well-being and inner beauty?
Debug: Found relevant node: mindfulness
Debug: Found relevant edge: ('inner_beauty', 'mindfulness', {'relationship': 'promotes'})
An error occurred: unhashable type: 'dict'
